In [9]:
## import
import lamindb as ln
import scanpy as sc

from scprint import scPrint
from scprint.dataloader import Collator, AnnDataCollator
from scdataloader.data import SimpleAnnDataset, Dataset
from scdataloader import Preprocessor
from torch.utils.data import DataLoader

from scdataloader.utils import load_genes

import numpy as np
import anndata as ad
from scipy.sparse import csr_matrix
import pandas as pd
import tqdm

from scvi.data import AnnDataManager
from scvi.data.fields import CategoricalObsField, LayerField
from scvi.dataloaders import AnnDataLoader
import scvi

from scib_metrics.benchmark import Benchmarker

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  %reload_ext autoreload


## preparing the model

In [11]:
model = scPrint.load_from_checkpoint('../../data/tensorboard/scprint_test/p1qh74w4/checkpoints/epoch=0-step=20212.ckpt')

In [3]:
ensembl = load_genes(['NCBITaxon:9606'])
ensembl['ensembl_gene_id'] = ensembl.index

## Loading the data

In [4]:
adata = sc.read(
    "data/pancreas_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539828",
)


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/X' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/X' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ml4ig1/miniconda3/envs/scprint

In [18]:
#adata = sc.read(
#    "data/lung_atlas.h5ad",
#    backup_url="https://figshare.com/ndownloader/files/24539942",
#)

/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/__categories/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarnin

In [ ]:
#adata = sc.read_h5ad('/home/ml4ig1/scprint/.lamindb/BljRloq1xjcxRNDpejzI.h5ad')

In [4]:
Preprocessor()(adata.copy())
# cannot preprocess (first of all not raw counts and also doesn't have standardized values)

ValueError: Data is not raw counts, please check layers, find raw data, or bypass with force_preprocess

In [5]:
# You need to use counts
adata.X = adata.layers['counts']
del adata.layers

In [6]:
# you need to have ensembl gene ids 
# # removing the other ones
var = adata.var.merge(ensembl.drop_duplicates('symbol').set_index('symbol', drop=False), left_index=True, right_index=True, how='inner').sort_values(by="ensembl_gene_id").set_index('ensembl_gene_id')
adata = adata[:, var['symbol']]
adata.var = var

# adding back the missing genes
unseen = set(ensembl.index) - set(adata.var.index)
# adding them to adata
emptyda = ad.AnnData(
    csr_matrix((adata.shape[0], len(unseen)), dtype=np.float32),
    var=pd.DataFrame(index=list(unseen)),
    obs=pd.DataFrame(index=adata.obs.index),
)
adata = ad.concat([adata, emptyda], axis=1, join="outer", merge="only")
# do a validation function
adata.uns["unseen_genes"] = list(unseen)
adata = adata[:, adata.var.sort_index().index]

# Add at least the organism you are working with
adata.obs['organism_ontology_term_id'] = "NCBITaxon:9606"
adata

AnnData object with n_obs × n_vars = 16382 × 70116
    obs: 'tech', 'celltype', 'size_factors', 'organism_ontology_term_id'
    var: 'symbol', 'ncbi_gene_id', 'biotype', 'description', 'synonyms', 'mt', 'ribo', 'hb', 'organism'
    uns: 'unseen_genes'

In [7]:
adataset = SimpleAnnDataset(adata, obs_to_output=['organism_ontology_term_id'])
col = Collator(organisms=["NCBITaxon:9606",], valid_genes=model.genes, max_len=1000, add_zero_genes=100)#mdataset.encoder['organism_ontology_term_id'])
dataloader = DataLoader(adataset, collate_fn=col, batch_size=32, num_workers=2, shuffle=False)

/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/pandas/core/algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


In [11]:
#dataloader = AnnLoader([adata], collate_fn=AnnDataCollator(max_len=5000, organisms=["NCBITaxon:9606",], org_to_id={'NCBITaxon:9606': 0}), batch_size=32, num_workers=1)

In [25]:
# dataloader = AnnDataLoader(adata_manager, collate_fn=SCVICollator(max_len=5000, organisms=["NCBITaxon:9606",], org_to_id={'NCBITaxon:9606': 0}), batch_size=32, num_workers=1)

In [12]:
from lightning.pytorch import Trainer

In [ ]:
trainer = Trainer(limit_predict_batches=1000)

INFO: GPU available: True (cuda), used: True
2024-02-07 09:22:38,107:INFO - GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
2024-02-07 09:22:38,110:INFO - TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
2024-02-07 09:22:38,113:INFO - IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
2024-02-07 09:22:38,116:INFO - HPU available: False, using: 0 HPUs


In [14]:
model.labels

[autoreload of scprint.model.loss failed: Traceback (most recent call last):
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/ml4ig1/Documents code/scPRINT/scprint/model/loss.py", line 201, in <module>
    class Similarity(nn.Module):
NameError: name 'nn' is not defined
]


['cell_type_ontology_term_id',
 'disease_ontology_term_id',
 'assay_ontology_term_id',
 'self_reported_ethnicity_ontology_term_id',
 'sex_ontology_term_id',
 'organism_ontology_term_id']

In [83]:
predictions = trainer.predict(model, dataloader, batch_embedding=['cell_type_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id'])

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2024-02-06 18:45:41,699:INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [84]:
predictions

[{'cell_emb': tensor([[-0.3292,  0.0631,  4.8392,  ..., -0.9505, -0.3213,  5.4364],
          [-0.3286,  0.0621,  4.8401,  ..., -0.9514, -0.3214,  5.4416],
          [-0.3289,  0.0626,  4.8396,  ..., -0.9511, -0.3214,  5.4394],
          ...,
          [-0.3287,  0.0624,  4.8397,  ..., -0.9514, -0.3214,  5.4413],
          [-0.3288,  0.0626,  4.8397,  ..., -0.9511, -0.3213,  5.4406],
          [-0.3292,  0.0629,  4.8394,  ..., -0.9507, -0.3213,  5.4371]])},
 {'cell_emb': tensor([[-0.3292,  0.0630,  4.8392,  ..., -0.9507, -0.3213,  5.4373],
          [-0.3265,  0.0583,  4.8455,  ..., -0.9488, -0.3219,  5.4492],
          [-0.3266,  0.0586,  4.8472,  ..., -0.9488, -0.3214,  5.4499],
          ...,
          [-0.3291,  0.0626,  4.8396,  ..., -0.9511, -0.3213,  5.4381],
          [-0.3292,  0.0631,  4.8392,  ..., -0.9504, -0.3213,  5.4368],
          [-0.3283,  0.0619,  4.8401,  ..., -0.9518, -0.3214,  5.4438]])},
 {'cell_emb': tensor([[-0.3291,  0.0628,  4.8394,  ..., -0.9508, -0.3213,  5

In [ ]:
adata.X.iloc[0].to_array()

In [85]:
for i in dataloader:
    break

In [86]:
i

{'x': tensor([[8.0570e+02, 3.7636e+01, 2.0824e+01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.4196e+03, 1.0631e+02, 8.7340e+01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.1688e+02, 1.0330e+02, 5.1928e+01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [8.5708e+02, 1.8352e+02, 6.7065e+01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.1383e+03, 1.0647e+03, 3.3937e+02,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.4196e+03, 4.8834e+02, 1.4376e+02,  ..., 0.0000e+00, 0.0000e+00,
          1.0020e+00]]),
 'genes': tensor([[ 3569, 12052,  9861,  ..., 25441,  9807, 27689],
         [ 3569,  1736, 24050,  ..., 21362,  8343, 26797],
         [ 3569,  1709,  1736,  ..., 28036, 20981,  6876],
         ...,
         [ 3569,  1736, 15005,  ..., 16754, 10537, 27996],
         [ 3569,  1736, 11387,  ..., 21687, 12838, 14328],
         [ 3569,  1736,  4758,  ...,   202, 13855,  6943]], dtype=t

In [ ]:
model

## results from running it on most famous methods

![](https://scib-metrics.readthedocs.io/en/stable/_images/72bdb33d77de77ce505e24fef1d700c2b91985b1d3fcce1c2b1dd7a35c8a647f.png)

In [ ]:
## load a random dataset

In [ ]:
## do scGPT finetuning task (reusing the helper functions I have) and the notebook that they provide

In [ ]:
## create a task/function for scPrint

In [ ]:
## try to do an embedding from the regulon / single cell type matrix outputted by scenic

In [ ]:
## push it to a function on BenGRN